In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets as dsets 
from torchvision import transforms

import numpy as np
from sklearn.datasets import make_multilabel_classification as make_data


In [ ]:
train = dsets.MNIST(root = './pytorch/softmax', train = True, download = True, transform = transforms.ToTensor())
val = dsets.MNIST(root = './pytorch/softmax', train = False, download = True, transform = transforms.ToTensor())

train_loader = DataLoader(dataset=train, batch_size=100, shuffle=True)
val_loader = DataLoader(dataset=val, batch_size=5000, shuffle=True)

In [2]:
class Softmax(nn.Module):

    def __init__(self, insize, outsize):
        super(Softmax, self).__init__()
        self.linear = nn.Linear(insize, outsize)

    def forward(self, x):
        yhat = self.linear(x)
        return yhat